<center><span style="background-color: rgb(251, 160, 38); font-size: 32px;">  <b>NUCLIO DIGITAL SCHOOL -</b> MASTER EN DATA SCIENCE  </span></center>
    
<br>
    
<center><a href = https://nuclio.school/wp-content/uploads/2019/10/nucleoDS-newBlack.png > <img src="https://nuclio.school/wp-content/uploads/2019/10/nucleoDS-newBlack.png" width=400 height=100><a/></center>

<hr style="height:5px;border-width:0;color:orange;background-color:orange">
    <center> <span style="font-size: 26px;"> KMeans with pipelines </span> </center>

+ Session: **KMeans with pipelines**
+ Module: **Unsupervised Learning**
+ Course: **Data Science Master 0921**
+ Professor: **Christa Santos**

<br>

<hr style="height:5px;border-width:0;color:orange;background-color:orange">

<a id = "toc"></a>
## Table of Contents
[1. Objective](#objectives)

[2. Import the main libraries](#import_modules)

[3. Import the data](#import_data)

[4. Exploratory Data Analysis (EDA)](#eda)

---> [4.1 Customers Dataset](#df1)

---> [4.2 Orders Dataset](#df2)

---> [4.3 Payments Dataset](#df3)

[5. Variable Creation](#variables)

[6. Final Dataset Creation](#join)

[7. Pipeline Creation](#skpipeline)

[8. Elbow curve](#elbow_curve)

[9. Customer segmentation with the "appropriate k"](#segmentation)

[10. RFM Model](#rfm)

[11. Summary](#summary)

[12. Conclusion](#conclusion)

<a id = "objectives"></a>
## 1. Objective
[Table of Contents](#toc)

In this notebook we are going to work with the dataset of the **Brazilian E-Commerce company called Olist.**

The company operates with a philosophy of ***SaaS (Software as a Service)***. Its basic product is a marketplace where stores with or without an online presence can sell their products to customers who visit their website.

![Data Model](https://i.imgur.com/HRhd2Y0.png)

Using the Olista dataset (around 100 thousand records) we are going to build a segmentation based on the KMeans algorithm and with a focus on constructing ***RFM variables (recency - frequency - monetary value).*** This It is a very common way of working in startups and online stores where little customer information is available and the retention / exchange of clicks is essential.

Our main objectives will be:
1. **Build business variables (based on segmentation)** from 3 different datasets and put them together correctly so as not to generate duplicate records.

2. **Learn to use the Sklearn Pipeline** to greatly speed up the transformation of the dataset and the creation of variables.

3. **Learn to implement our own Transformers** that can be used inside the Pipelines (for example to eliminate outliers).

4. Use the elbow diagram to determine the **"optimal"** number of centroids for the KMeans model.

5. **Summarize the information of our clusters in a more friendly format** using Pandas.

<a id = "import_modules"></a>
## 2. Import the main libraries
[Table of Contents](#toc)

In this section of the kernel we are going to load the main libraries that we are going to use in our notebook during the implementation of the **KMeans algorithm**.

In [ ]:
# silence warnings
import warnings
warnings.filterwarnings("ignore")

# operating system
import os

# time calculation to track some processes
import time

# numeric and matrix operations
import numpy as np
import pandas as pd

# loading ploting libraries
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
%matplotlib inline

# python core library for machine learning and data science
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.cluster import KMeans

RANDOM_STATE = 175
PATH_FOLDER = os.path.join(os.getcwd(), "data\\olist_datasets")

In [ ]:
import sklearn
print(sklearn.__version__)

In [ ]:
PATH_FOLDER

<a id = "import_data"></a>
## 3. Import the data
[Table of Contents](#toc)

In this section of the kernel we are going to load the main datasets that we are going to use for our segmentation.

You can **add other datasets or external data** to drill down or experiment with the KMeans algorithm.

In [ ]:
PATH_CUSTOMERS = os.path.join(PATH_FOLDER, 'olist_customers_dataset.csv')

customer_df = pd.read_csv(PATH_CUSTOMERS)

In [ ]:
PATH_ORDERS = os.path.join(PATH_FOLDER, 'olist_orders_dataset.csv')

orders_df = pd.read_csv(PATH_ORDERS)

In [ ]:
PATH_PAYMENTS = os.path.join(PATH_FOLDER, 'olist_order_payments_dataset.csv')

payments_df = pd.read_csv(PATH_PAYMENTS)

<a id = "eda"></a>
## 4. Exploratory Data Analysis (EDA)
[Table of Contents](#toc)

In the EDA section we will make **a first approximation to our data** to see its composition and what variables we have at our disposal.

<a id = "df1"></a>
### 4.1 Customers Dataset
[Table of Contents](#toc)

Quick EDA on the **customer dataset.**

In [ ]:
def report_df(df, verbose = True):
    '''
    Makes a simple report on the supplied DataFrame.
    '''
    print(df.info(verbose = verbose))
    total_nulls = df.isnull().sum().sum()
    print()
    print(f"We have a total of {total_nulls} nulls")

In [ ]:
report_df(customer_df)

customer_df.head()

In [ ]:
customer_id = customer_df["customer_id"].nunique()
customer_unique_id = customer_df["customer_unique_id"].nunique()

print(f'''We have a total of {customer_unique_id} unique customers, for a total of {customer_id} orders (in Olist's schema, they specify that the unique id is customer_unique_id and that customer_id is an id that is generated in each purchase and therefore in practice it is the same as an order).
This implies a ratio of {round(customer_id/customer_unique_id, 2)} orders per customer.
''')

In [ ]:
# create a groupby DataFrame by customer city
city_pivot_ = customer_df.groupby(["customer_state", "customer_city"]).size()\
.sort_values(ascending = False).to_frame().reset_index().rename(columns = {0: "clients_per_city"})

city_pivot_["cumsum_by_city"] = (city_pivot_["clients_per_city"]/city_pivot_["clients_per_city"].sum()).cumsum()

city_pivot_["pct_cities"] = (1/city_pivot_.shape[0])
city_pivot_["pct_cities"] = city_pivot_["pct_cities"].cumsum()

In [ ]:
# create a groupby DataFrame by customer state
state_pivot_ = customer_df.groupby("customer_state").size()\
.sort_values(ascending = False).to_frame().reset_index().rename(columns = {0: "clients_per_state"})

state_pivot_["cumsum_by_state"] = (state_pivot_["clients_per_state"]/state_pivot_["clients_per_state"].sum()).cumsum()

state_pivot_["pct_state"] = (1/state_pivot_.shape[0])
state_pivot_["pct_state"] = state_pivot_["pct_state"].cumsum()

In [ ]:
#----------------------
# Plotting part

# Instantiate the figure
fig = plt.figure(figsize = (10, 15))
ax1, ax2, ax3 = fig.subplots(nrows = 3, ncols = 1)

# get the data
# concate a zero before the list, so that all curves start at origin
x1_values = [0] + list(city_pivot_.index)
y1_values = [0] + list(city_pivot_["cumsum_by_city"])

x2_values = [0] + list(state_pivot_["cumsum_by_state"])
y2_values = [0] + list(state_pivot_["cumsum_by_state"])

x3_values_city = [0] + list(city_pivot_["pct_cities"])
y3_values_city = [0] + list(city_pivot_["cumsum_by_city"])

x3_values_state = [0] + list(state_pivot_["pct_state"])
y3_values_state = [0] + list(state_pivot_["cumsum_by_state"])

# plot the values and set for every subplot a title
ax1.plot(y1_values)
ax1.title.set_text("Percentage of Accumulated Customers by City")

ax2.plot(y2_values, color = "green", alpha = 0.5)
ax2.title.set_text("Percentage of Accumulated Customers by State")

ax3.plot(x3_values_city, y3_values_city, label = "Percentage of Accumulated Customers by City")
ax3.plot(x3_values_state, y3_values_state, label = "Percentage of Accumulated Customers by State", 
         color = "green", alpha = 0.5)
ax3.title.set_text("Percentage of Accumulated Customers by State and City")

ax3.legend()

# create a title for the figure
fig.suptitle('Cumulative distribution of customers by City and State (absolute and relative)', fontsize = 16);

<a id = "df2"></a>
### 4.2 Orders Dataset
[Table of Contents](#toc)

Quick EDA on the **order dataset.**

In [ ]:
report_df(orders_df)

orders_df.head(3).T

In [ ]:
orders_df["order_status"].value_counts()

In [ ]:
# We have nulls in the dates, we will either have to eliminate these nulls or assigning a date to them.
orders_df.isnull().sum()

In [ ]:
order_id = orders_df["order_id"].nunique()
customer_id_orders = orders_df["customer_id"].nunique()

# We make sure that all orders have their corresponding customer in the customers table
assert (set(orders_df["customer_id"]) ^ set(customer_df["customer_id"])) == set()
assert (set(customer_df["customer_id"]) ^ set(orders_df["customer_id"])) == set()

print(f"We have a total of {order_id} orders")
print(f"We have a total of {customer_unique_id} unique customers (from the customer dataset)")
print(f"The relation of orders by clients is of {round(order_id/customer_unique_id, 2)}")

<a id = "df3"></a>
### 4.3 Payments Dataset
[Table of Contents](#toc)

Quick EDA on the **payment dataset.**

In [ ]:
report_df(payments_df)

payments_df.head()

In [ ]:
# payment sequential: a customer may pay an order with more than one payment method. If he does so, a sequence will be created to accommodate all payments.

payments_df['payment_sequential'].value_counts()

In [ ]:
order_id_pay = payments_df["order_id"].nunique()
print(f"We have a total of {order_id_pay} unique orders in the payments table.")

In [ ]:
# We make sure that all orders have their corresponding customer in the customers table
len(set(orders_df["order_id"]))

In [ ]:
set(orders_df["order_id"]) ^ set(payments_df["order_id"])

In [ ]:
orders_df[orders_df["order_id"] == "bfbd0f9bdef84302105ad712db648a6c"]

<a id = "variables"></a>
## Feature Engineering
[Table of Contents](#toc)

We have analyzed the 3 key datasets with which we are going to work.

In this section of the notebook, **we will add the dataset and we will generate new variables** so that later in the next section we will put it together in one dataset and do the segmentation.

In [ ]:
# We do a groupby by order and calculate some basic metrics like max, min, mean and the like.
# We have seen that a large part of the payments are unique, therefore they will coincide in most, but in the other cases
# they will provide us with useful information about the client.

aggregated_payments = payments_df.groupby('order_id').agg(
    max_pay = ('payment_value', 'max'), 
    min_pay = ('payment_value', 'min'),
    mean_pay = ('payment_value', 'mean'),
    total_pay = ('payment_value', 'sum'),
    max_seq = ('payment_sequential', 'max')
)

In [ ]:
aggregated_payments[aggregated_payments["max_seq"] == 3]

In [ ]:
payments_df.shape

In [ ]:
aggregated_payments.shape

In [ ]:
orders_df.shape

In [ ]:
orders_with_payments = pd.merge(orders_df, aggregated_payments, on = 'order_id')

In [ ]:
orders_with_payments.shape

In [ ]:
report_df(orders_with_payments)

In [ ]:
orders_with_payments.head(3).T

In [ ]:
orders_with_payments.set_index('order_id', inplace = True)

In [ ]:
# we convert the dates to the dates format since until now they were objects and they did not allow us to do any transformation

orders_with_payments['order_purchase_timestamp'] =\
pd.to_datetime(orders_with_payments['order_purchase_timestamp'], format = '%Y-%m-%d')

orders_with_payments['order_delivered_customer_date'] =\
pd.to_datetime(orders_with_payments['order_delivered_customer_date'], format = '%Y-%m-%d')

orders_with_payments['order_estimated_delivery_date'] =\
pd.to_datetime(orders_with_payments['order_estimated_delivery_date'], format = '%Y-%m-%d')

In [ ]:
report_df(orders_with_payments)

In [ ]:
# We create useful variables by extracting the order information.
# For example: time since the last purchase can be very useful for our client to detect recent customers 
# and combined with the total orders variable it can be very valuable to segment very loyal customers

orders_with_payments['last_purchase'] = orders_with_payments['order_purchase_timestamp'].max()

orders_with_payments['time_since_last_purchase'] =\
orders_with_payments['last_purchase'] - orders_with_payments['order_purchase_timestamp']

orders_with_payments['delivery_time'] =\
orders_with_payments['order_delivered_customer_date'] - orders_with_payments['order_purchase_timestamp']

orders_with_payments['delay'] =\
orders_with_payments['order_delivered_customer_date'] - orders_with_payments['order_estimated_delivery_date']

In [ ]:
orders_with_payments.head().T

<a id = "join"></a>
## Final Dataset Creation
[Table of Contents](#toc)

In [ ]:
df_final = pd.merge(customer_df, orders_with_payments, on = "customer_id")

In [ ]:
df_final.set_index("customer_unique_id", inplace = True)

lc = [
    'max_pay',
    'min_pay',
    'mean_pay',
    'total_pay',
    'max_seq',
    'time_since_last_purchase',
    'delivery_time',
    'delay'
]

df_final = df_final[lc]

# We extract the days from the timedelta variable that we had previously calculated.
df_final["time_since_last_purchase"] = df_final["time_since_last_purchase"].dt.days
df_final["delivery_time"] = df_final["delivery_time"].dt.days
df_final["delay"] = df_final["delay"].dt.days

In [ ]:
df_final.head().T

In [ ]:
report_df(df_final)

df_final.sample(3).T

In [ ]:
# We have null values and we will have to impute them correctly in our pipeline.
df_final.isnull().sum()

<a id = "skpipeline"></a>
## Pipeline Creation
[Table of Contents](#toc)

At this point we have to do the last transformations of our dataframe as well as create the last variables for the **RFM model.**

One way to do this is to use the sklearn pipeline to automate all these steps, but for this we have to implement our own ***"Transformers."***

A ***Transformer*** in sklearn is not just another class that has the **fit, transform and fit_transform** method implemented (which can perform any transformation on the dataset that you pass to it).

However, in practice we are only going to implement one method, the **transform**, because the **fit** (when we inherit from ***TransformerMixin***) only has to return the ***self*** and the **fit_transform** is to be created only.

We also inherit from the ***BaseEstimator*** to have access to the method of ***get_params() and set_params()*** although in practice we are not going to use it.

In [ ]:
#df_final_backup = df_final.copy(deep=True)

The ***outliers*** can distort our KMeans a lot, being able to create groups of only 1 client.

To prevent this from happening, we have to remove any possible ***outlier.*** and then standardize or normalize our data.

Next we are going to implement our own ***OutlierFilter***

In [ ]:
class OutlierFilter(BaseEstimator, TransformerMixin):
    '''
    Class that filters outliers using np.quantile ()
    The quantiles to filter as well as the columns to filter are the parameters of the class.
    '''
    
    def __init__(self, q, col_to_filter):
        self.q = q
        self.col_to_filter = col_to_filter
        
    def fit(self, X, y = None):
        return self
    
    def transform(self, X, y = None):
        '''
        The method considers that client outlier that is outlier in all the columns that you pass to it.
        That is to say: if you have to filter the amount and number of orders, you will only eliminate those customers
        which are outlier both in amount and number of orders. If you are outlier in amount but not in order
        it will not be filtered from the dataset.
        '''
        
        # empty list
        criteria_list = []
        
        # we add clients that are outliers to the list
        for col in self.col_to_filter:
            criteria = X[col] < np.quantile(X[col], q = self.q)
            criteria_list.append(criteria)
            
        # if there is more than 1 column
        if len(self.col_to_filter) > 1:
            
            # we create the global criterion: that is, outlier in all columns
            global_criteria = criteria_list[0]
            
            for criteria in criteria_list[1:]:
                global_criteria = global_criteria & criteria
                
        else:
            global_criteria = criteria_list[0]
            
        # we filter our dataframe
        X = X[global_criteria]
        
        # we save the index as a class parameter because otherwise we would lose it.
        self.index = X.index
        
        return X

We have reached this point and we have everything ready to build our pipeline.

The steps you are going to perform are:

1. Use the ***KNNImputer***, which is based on the same notion of ** distance metrics ** to impute null values ​​based on the most similar clients.

2. Since it returns an array in the first step, we use our own ***Transformer*** to convert to DataFrame.

3. We create variables at the customer level. **SO OUR OUTPUT DATAFRAME WILL BE SMALLER.**

4. We filter the outliers with our own ***Transformer***. **SO OUR OUTPUT DATAFRAME WILL BE SMALLER.**

5. We standardize the values, using ***StandardScaler***.

6. We make a fit with KMeans to calculate the **inertia** of the groups (the dispersion of the data to the centroid).

***Elbow Curve*** Technique: we do all this in a loop because we want to see when there is a sudden change in inertia and therefore **increasing the number of centroids further does not pay off because the marginal gain it is very small.**

In [ ]:
CALCULATE_ELBOW = True

In [ ]:
# separamos el pipeline del a loop, para no tener que volver a hacer los primeros 5 pasos para cada k de la loop
pipe = Pipeline(steps = [
    ("Imputer", KNNImputer()),
    ("ArrayToDataFrame", ArrayToDataFrame(columns, index = index)),
    ("FeatureGenerator", FeatureGenerator()),
    ("OutlierFilter", OutlierFilter(q = 0.99, col_to_filter = ["amount", "max_delay"])),
    ("StandardScaler", StandardScaler())
])

In [ ]:
df_scaled_transformed_no_outliers = pipe.fit_transform(df_final)

<a id = "elbow_curve"></a>
# Elbow Curve
[Table of Contents](#toc)

In this section we are going to visualize our ***Elbow Curve*** and we will look for the inflection point that will be our number of centroids.

In [ ]:
# df_final = pd.read_csv
df_final.isnull().sum()

In [ ]:
if CALCULATE_ELBOW:
    fig = plt.figure(figsize = (16, 8))
    ax = fig.add_subplot()

    x_values = list(sse.keys())
    y_values = list(sse.values())

    ax.plot(x_values, y_values, label = "Inertia / dispersion of clusters")
    fig.suptitle("Variation of the dispersion of the clusters as a function of the k", fontsize = 16);

<a id = "segmentation"></a>
## Customer Segmentation with the "appropriate k"
[Table of Contents](#toc)

Now that we have determined the correct number of centroids we can fit our pipeline with the appropriate ***k.***

Since we are going to carry out our segmentation with KMeans and we are going to supply them with the variables of our interest, sometimes, KMeans is known as **unsupervised but guided segmentation***. Guided because somehow the data scientist tells (guides) it to discriminate using some variables and not others.

In [ ]:
pipe = Pipeline(steps = [
    ("Imputer", KNNImputer()), 
    ("ArrayToDataFrame", ArrayToDataFrame(columns, index = index)),
    ("FeatureGenerator", FeatureGenerator()),
    ("OutlierFilter", OutlierFilter(q = 0.99, col_to_filter = ["amount", "max_delay"])),
    ("StandardScaler", StandardScaler()),
    ("Clustering", KMeans(n_clusters = 5))
])

In [ ]:
df_final.shape

In [ ]:
pipe.fit(df_final)

A very interesting part of pipelines is that we can filter it (just like a python list) and use only part of the steps that we have implemented.

This is very useful because in our case, when we are going to predict (assign each client its centroid), we want to impute the nulls, create the necessary variables and standardize (step 1, 3 and 5 of the pipe), but **not filter outliers** (all clients must have a group). If we predict with the entire pipeline, some clients will not be assigned to any group.

In [ ]:
# we create a scaled dataframe with steps 1, 3 and 5
X_processed = pipe[:3].transform(df_final)
X_scaled = pipe["StandardScaler"].transform(X_processed)

# We lose customers by step 3: customers with more than 1 order or payment, it ends up being grouped in 1 single record.
X_scaled.shape

In [ ]:
# We do the predict, in this case we will have their centroid / cluster for each client.
labels = pipe["Clustering"].predict(X_scaled)

In [ ]:
pipe["Clustering"]

In [ ]:
# we assign the centroids to the processed DataFrame.
# IF WE DO IT TO SCALING THE NUMBERS WILL LOSE THEIR MEANING TO SCALE AND WILL BE MORE DIFFICULT TO INTERPRET.
X_processed["cluster"] = labels

In [ ]:
X_processed.shape

In [ ]:
# We visualize our groups based on the variables of the RFM model, to see how they have been.
selected_columns = ['n_orders', 'amount', 'last_purchase']

sns.pairplot(X_processed, vars = selected_columns, hue = 'cluster');

<a id = "rfm"></a>
## RFM Model
[Table of Contents](#toc)

In this section we are going to see how a segmentation based on 3 key indicators of: ***retention, frequency and monetary value (RFM) can be implemented very easily.***

In [ ]:
# we calculate the RFM indicators
QUANTILES = 5
X_processed['recency'] = pd.qcut(X_processed['last_purchase'], q = QUANTILES, labels = range(QUANTILES))
X_processed['frequency'] = (X_processed['n_orders'] > 1).astype(int)
X_processed['monetary_value'] = pd.qcut(X_processed['amount'], q = QUANTILES, labels = range(QUANTILES))

In [ ]:
X_processed.pivot_table(index = 'recency', values = ['last_purchase'], aggfunc = [len, np.mean])

In [ ]:
X_processed.pivot_table(index = 'monetary_value', values = ['amount'], aggfunc = [len, np.mean])

In [ ]:
X_processed.pivot_table(index = 'frequency', values = ['n_orders'], aggfunc = [len, np.mean])

In [ ]:
rm_df = pd.crosstab(X_processed['recency'], X_processed['monetary_value'])

In [ ]:
rm_df

In [ ]:
sns.heatmap(rm_df, cmap = 'RdYlGn', linewidths = 0.5)

In [ ]:
X_processed[['recency', 'frequency', 'monetary_value', "n_orders"]].\
groupby(['recency', 'frequency', 'monetary_value']).agg('count')

<a id = "summary"></a>
# Summary
[Table of Contents](#toc)

The last step, used once we have done our complete segmentation, is to create a ***summary tab*** of each group with the main business variables or with those that have not been used in the segmentation to periodically monitor the groups or to send as a document to the rest of the company's departments.

In [ ]:
summary_df = pd.DataFrame()
summary_df

In [ ]:
for i, col in enumerate(["amount", "n_orders", "last_purchase", "mean_delay"]):
    summary_data = X_processed[["cluster", col]].groupby("cluster").describe().T[1:]
    summary_df = summary_df.append(summary_data)

In [ ]:
summary_data

In [ ]:
summary_df.head(10)

In [ ]:
summary_df.head(7)

In [ ]:
# we generate our multiindex
out_index = [
     "Monetary",
     "Loyalty",
     "Loyalty",
     "Logistics"
]

inner_index = [
     "Amount",
     "Nr. Of purchases",
     "Last purchase",
     "Delays"
]

statistics = ["Average", "Deviation", "Minimum", "Perc. 25", "Perc. 50", "Perc. 75", "Maximum"]

new_multi_index = []

for oi, ii, in zip(out_index, inner_index):
    for es in statistics:
        new_multi_index.append((oi, ii, es))
        
new_multi_index

In [ ]:
def generate_multiindex(list_of_tuples, names):
    return pd.MultiIndex.from_tuples(list_of_tuples, names = names)

In [ ]:
names = ["Indicator Group", "Indicator", "Statistic"]
index_df = generate_multiindex(new_multi_index, names)
summary_df.set_index(index_df, inplace = True)

In [ ]:
size_clusters = X_processed.groupby("cluster").size().to_frame().T
size_clusters.set_index(generate_multiindex([("General", "Cluster", "Size")] , names), inplace = True)

In [ ]:
summary_df = size_clusters.append(summary_df)

In [ ]:
summary_df

<a id = "conclusion"></a>
# Conclusion
[Table of Contents](#toc)

Unsupervised algorithms are **very powerful** tools that any data scientist should have on hand. Knowing how they work and implementing them correctly can allow you to **extract valuable information and make better business decisions**. We have seen different algorithms and unsupervised techniques (***KMeans and RFM model***) that allow us to create **homogeneous and actionable groups** of clients and thus improve the company's indicators.

In addition to this, we have learned to use sklearn pipelines to **fully automate data processing** as well as implement our own ***Transformers*** that can be used within the pipeline.